In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Read the CSV file
data = pd.read_csv('/content/drive/MyDrive/NLP/YorumandAI.csv')
data

,Unnamed: 0,index,Review,Class
0,0,1475,"Ortamı güzel, çalışanların hizmet anlayışı çok...",1.0
1,1,692,Tost soğumuştu ve eklenmesini isteyip ödemesin...,0.0
2,2,100,"Yağmur, kar ve çamur demeden gece yarısı servi...",0.0
3,3,1826,Müşterilerinin beklentilerini karşılamadığını ...,1.0
4,4,1761,"Yediklerimizin sıcaklığı, lezzeti ve porsiyon ...",1.0
...,...,...,...,...
2042,2042,1130,Sunulan yemeklerin tadı çok kötüydü. Tavuk yem...,1.0
2043,2043,1294,Müşteriler tarafından gösterilen övgüye layık ...,1.0
2044,2044,860,Ellerinize sağlık.,0.0
2045,2045,1459,Siparişimi aldıktan sonra çok kısa bir süre iç...,1.0


In [ ]:
data=data.drop(["Unnamed: 0", "index"], axis=1)

In [ ]:

# Shuffle the data randomly
data = data.sample(frac=1, random_state=42)

# Calculate the split indices
split_index = int(0.6 * len(data))

# Split the data into train and test sets
train_data = data.iloc[:split_index]
test_data = data.iloc[split_index:]

# Reset the index for both dataframes
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

# Display the shape of train and test dataframes
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

Train data shape: (1228, 2)
Test data shape: (819, 2)


In [ ]:
train_data


,Review,Class
0,Yediklerimiz sıcak olarak sunulmuyordu. Kalite...,1.0
1,Yani yanlis anlamayin servis ve hiz cidden iyi...,0.0
2,Karışık 2 çok iyiydi 20 dk civarı geldi teşekk...,0.0
3,Yemeklerin çoğu çok tuzlu ve berbat pişmişti. ...,1.0
4,Her şey iyi güzel hoş ama rokaları koyarken ko...,0.0
...,...,...
1223,Su boregi cok lezzetliydi. Yaninda havuc dilim...,0.0
1224,Hamburgeriniz iyi güzel ama ne ranch sos ne ac...,0.0
1225,Gerçekten bekledigimden daha iyi uzun zamandır...,0.0
1226,Hizmet ve ürün kalitelerini harika buldum.,0.0


In [ ]:
test_data

,Review,Class
0,Çok beğendim gayet güzeldi lahmacunlar sıcak v...,0.0
1,Müşterilerinin memnuniyetini ön planda tutan b...,1.0
2,Lezzetliydi fakat tuzsuz.,0.0
3,Güzel.,0.0
4,"Hız, servis, lezzet çok güzel çok hızlıydı iki...",0.0
...,...,...
814,"Müşterilerin, yemek siparişi verdikleri mekand...",1.0
815,Sunulan yemeklerin her birinde farklı bir lezz...,1.0
816,"Müşterilere lezzetli, taze ve kaliteli yemekle...",1.0
817,"Bu restoranın lezzetli yemekleri, kaliteli hiz...",1.0


In [ ]:
train_texts = train_data['Review'].tolist()
train_labels = train_data['Class'].tolist()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-128k-uncased', do_lower_case=True)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
train_labels = torch.tensor(train_labels)

In [ ]:
#PyTorch datasets
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              train_labels)

In [ ]:
#data loader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-turkish-128k-uncased",
    num_labels =2,
    output_attentions = False,
    output_hidden_states = False,
)


Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassificatio

In [ ]:
# Set device and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import torch.nn as nn

num_epochs = 6
criterion = nn.BCEWithLogitsLoss()

model.train()
for epoch in range(num_epochs):
    total_loss = 0

    for batch in train_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits.squeeze(1)

        labels_one_hot = torch.zeros_like(logits)
        labels_one_hot.scatter_(1, labels.unsqueeze(1).to(torch.int64), 1)

        loss = criterion(logits, labels_one_hot)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {average_loss:.4f}")


Epoch 1/6 - Loss: 0.4220
Epoch 2/6 - Loss: 0.0763
Epoch 3/6 - Loss: 0.0239
Epoch 4/6 - Loss: 0.0133
Epoch 5/6 - Loss: 0.0091
Epoch 6/6 - Loss: 0.0068


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/NLP/BERT_turk/model_y.pth')

In [ ]:
model.save_pretrained("/content/drive/MyDrive/NLP/Model_fine")


In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/NLP/Model_fine")


('/content/drive/MyDrive/NLP/Model_fine/tokenizer_config.json',
 '/content/drive/MyDrive/NLP/Model_fine/special_tokens_map.json',
 '/content/drive/MyDrive/NLP/Model_fine/vocab.txt',
 '/content/drive/MyDrive/NLP/Model_fine/added_tokens.json')

In [ ]:
test_data

,Review,Class
0,Çok beğendim gayet güzeldi lahmacunlar sıcak v...,0.0
1,Müşterilerinin memnuniyetini ön planda tutan b...,1.0
2,Lezzetliydi fakat tuzsuz.,0.0
3,Güzel.,0.0
4,"Hız, servis, lezzet çok güzel çok hızlıydı iki...",0.0
...,...,...
814,"Müşterilerin, yemek siparişi verdikleri mekand...",1.0
815,Sunulan yemeklerin her birinde farklı bir lezz...,1.0
816,"Müşterilere lezzetli, taze ve kaliteli yemekle...",1.0
817,"Bu restoranın lezzetli yemekleri, kaliteli hiz...",1.0


In [ ]:
import torch
from transformers import BertTokenizer

def preprocess_data(data):
    # Initialize the tokenizer
    tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-128k-uncased',num_labels=2)

    # Tokenize text and convert to input tensors
    input_ids = []
    attention_masks = []
    labels = []

    for index, row in data.iterrows():
        text = row['Review']
        label = row['Class']

        # Tokenize text
        encoded_inputs = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Extract input IDs, attention masks, and labels
        input_ids.append(encoded_inputs['input_ids'].squeeze())
        attention_masks.append(encoded_inputs['attention_mask'].squeeze())
        labels.append(label)

    # Convert lists to tensors
    input_ids = torch.stack(input_ids)
    attention_masks = torch.stack(attention_masks)
    labels = torch.tensor(labels)

    return input_ids, attention_masks, labels

# Preprocess test data and convert to tensors
test_input_ids, test_attention_masks, test_labels = preprocess_data(test_data)

# Create test dataset
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)


# Create test data loader
batch_size = 32  # Choose an appropriate batch size
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# Load the saved model state dictionary
model.load_state_dict(torch.load('/content/drive/MyDrive/NLP/BERT_turk/model_y.pth'))


<All keys matched successfully>

In [ ]:
# Set device for testing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(128000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
# Set the model to evaluation mode
model.eval()

# Initialize variables for evaluation
test_loss = 0
correct_predictions = 0
total_predictions = 0

# Disable gradient calculation during evaluation
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Perform forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Convert logits to probabilities using sigmoid activation
        probabilities = torch.sigmoid(logits)

       # Round probabilities to obtain predicted labels (0 or 1)
        predicted_labels = torch.round(probabilities)[:, 1]

# Squeeze the labels tensor to remove the extra dimension
        labels_squeezed = labels.squeeze(dim=-1)

# Expand the dimensions of the labels tensor
        expanded_labels = labels_squeezed.unsqueeze(dim=1)

# Reshape predicted_labels to match the shape of expanded_labels
        reshaped_predicted_labels = predicted_labels.view(-1, 1)

# Calculate the loss
        loss = criterion(reshaped_predicted_labels, expanded_labels)


        # Update evaluation metrics
        test_loss += loss.item()
        total_predictions += labels.size(0)
        # Calculate the number of correct predictions
        correct_predictions += (predicted_labels == labels_squeezed).sum().item()

# Calculate average loss and accuracy
average_loss = test_loss / len(test_loader)
accuracy = correct_predictions / total_predictions

# Print the evaluation results
print(f"Test Loss: {average_loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Test Loss: 0.5083
Test Accuracy: 0.9976


In [ ]:
# Set the model to evaluation mode
model.eval()

# Initialize variables for evaluation
test_loss = 0
correct_predictions = 0
total_predictions = 0
predicted_labels_list = []

# Disable gradient calculation during evaluation
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Perform forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Convert logits to probabilities using sigmoid activation
        probabilities = torch.sigmoid(logits)

        # Round probabilities to obtain predicted labels (0 or 1)
        predicted_labels = torch.round(probabilities)[:, 1]

        # Append predicted labels to the list
        predicted_labels_list.extend(predicted_labels.tolist())

        # Squeeze the labels tensor to remove the extra dimension
        labels_squeezed = labels.squeeze(dim=-1)

        # Expand the dimensions of the labels tensor
        expanded_labels = labels_squeezed.unsqueeze(dim=1)

        # Reshape predicted_labels to match the shape of expanded_labels
        reshaped_predicted_labels = predicted_labels.view(-1, 1)

        # Calculate the loss
        loss = criterion(reshaped_predicted_labels, expanded_labels)

        # Update evaluation metrics
        test_loss += loss.item()
        total_predictions += labels.size(0)
        # Calculate the number of correct predictions
        correct_predictions += (predicted_labels == labels_squeezed).sum().item()

# Calculate average loss and accuracy
average_loss = test_loss / len(test_loader)
accuracy = correct_predictions / total_predictions

# Add predicted labels as a new column to the test dataframe
test_data['predicted_label'] = predicted_labels_list

# Print the evaluation results
print(f"Test Loss: {average_loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Test Loss: 0.5028
Test Accuracy: 0.9976


In [ ]:
test_data

,Review,Class,predicted_label
0,Merhabalar müşteri memnuniyetine çok önem verm...,0.0,0.0
1,Çeşitli yemek seçenekleri sunan lezzetli yemek...,1.0,1.0
2,Gayet başarılı. Kayseri usulu değil ev usulu y...,0.0,0.0
3,Müşterilerin beklentilerinin neredeyse tümünü ...,1.0,1.0
4,Çok iyi.,0.0,0.0
...,...,...,...
815,Sadece çoban salatanın soğanları büyük geldi o...,0.0,0.0
816,"Patates biraz soguk geldi, onun disinda her se...",0.0,0.0
817,"Uygun fiyatlarına rağmen, yiyeceklerin lezzeti...",1.0,1.0
818,"İnanılmaz derecede yüksek fiyatlarıyla, bu res...",1.0,1.0
